### Data visualization

In [ ]:
import math

import matplotlib.pyplot as plt
import matplotlib as mpl

# plot several images
def plot_instances(size, instances, labels, figure_size=(8, 10), set_title=False, images_per_row=10):
  rows = math.ceil(len(instances) / images_per_row) 
  fig, axes = plt.subplots(nrows=rows, ncols=images_per_row, figsize=figure_size)
  if rows == 1:
    for index, instance in enumerate(instances):
      plt.subplot(1, images_per_row, index+1)
      plt.imshow(instance, cmap='binary')
      plt.axis('off')
      plt.title(label=labels[index])
  else:
    for i in range(rows):
      for j in range(images_per_row):
        if i*images_per_row+j >= len(instances):
          empty_image = np.zeros((size, size))
          axes[i,j].imshow(empty_image, cmap='binary')
        else:
          axes[i,j].imshow(instances[i*images_per_row+j].reshape(size, size), cmap='binary')
          if set_title:
            axes[i,j].title.set_text(labels[i*images_per_row+j])
        axes[i,j].axis('off')

# another way
def plot_instances(n_rows, n_cols, instances, set_title=False, labels=[], figure_size=(10, 8)):
  fig, axs = plt.subplots(n_rows, n_cols, figsize=figure_size)
  for i, ax in enumerate(axs.flat):
    ax.imshow(instances[i], cmap='binary')
    ax.set_axis_off()
    if set_title:
      ax.set_title(labels[i])

In [ ]:
# heatmap
import seaborn as sns

sns.heatmap(conf_mx)
plt.show()

####################
plt.matshow(conf_mx)
plt.show()

### Other tools

In [ ]:
from sklearn.model_selection import train_test_split

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

### Missing Value

In [ ]:
# find the numbers of missing value
housing.isnull().sum()

In [ ]:
# drop the rows with null values
drop_1 = housing.dropna(subset=["total_bedrooms"])

# drop thr colomns with too much null values that we don't like
drop_2 = housing.drop('total_bedrooms', axis=1)

# replace with specific values, don't forget to save for later use in the test set
median = housing_new['total_bedrooms'].median()
housing_new['total_bedrooms'].fillna(median, inplace=True)

### Text Categories to numbers

 

In [ ]:
# more suitable for ordinal categories like 'bad', 'good'
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
# take care of [[ ]]
housing_cat_encoded = ordinal_encoder.fit_transform(housing[['ocean_proximity']])
# check categories
ordinal_encoder.categories_

In [ ]:
# more suitable for nominal categories like 'typeA', 'typeB'
from sklearn.preprocessing import OneHotEncoder

nominal_encoder = OneHotEncoder()
# return a SciPy sparse matrix
housing_cat_1hot = nominal_encoder.fit_transform(housing[['ocean_proximity']])
# or use this code
# nominal_encoder = OneHotEncoder(sparse=False)

# convert sparse matrix to array
housing_cat_1hot.toarray()
# check categories
nominal_encoder.categories_

### Data Augmentation

In [ ]:
# shift image
from scipy.ndimage.interpolation import shift

def shift_image(image, dx, dy):
    image = image.reshape((28, 28))
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image.reshape([-1])

image = X_train[1000]
shifted_image_down = shift_image(image, 0, 5) # down 5 pixels
shifted_image_left = shift_image(image, -5, 0) # left 5 pixels

### Feature Scaling
As with all the transformations, it is important to fit the scalers to the training data only, not to the full dataset (including the test set). Only then can you use them to transform the training set and the test set (and new data).

In [ ]:
# min-max scaling
from sklearn.preprocessing import MinMaxScaler

minmax_scaler = MinMaxScaler()
# housing_scale = minmax_scaler.fit(housing_new[['median_income']])
housing_scale = minmax_scaler.fit_transform(housing_new[['median_income']])

# standardization does not bound values to a specific range, like '0-1'
# standardization is 0 mean and unit variance
# standardization is less affected by outliers
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()

### Other Tools for Data Preparetion

In [ ]:
col_names = "total_rooms", "total_bedrooms", "population", "households"
rooms_ix, bedrooms_ix, population_ix, households_ix = [housing.columns.get_loc(c) for c in col_names] # get the column indices

#### Dimensionality Reduction

In [ ]:
# PCA (Principal Component Analysis)
'''
PCA identifies the axis that accounts for the largest amount of variance in
the training set.

Reducing dimensionality does cause some information loss (just like compressing an
image to JPEG can degrade its quality), so even though it will speed up training, it
may make your system perform slightly worse. It also makes your pipelines a bit
more complex and thus harder to maintain. So, if training is too slow, you should
first try to train your system with the original data before considering using
dimensionality reduction. In some cases, reducing the dimensionality of the training
data may filter out some noise and unnecessary details and thus result in higher
performance, but in general it won’t; it will just speed up training.

PCA assumes that the dataset is centered around the origin. As we will see, Scikit-
Learn’s PCA classes take care of centering the data for you. If you implement PCA
yourself, or if you use other libraries, don’t forget to center the data first.
'''

from sklearn.decomposition import PCA
# Scikit-Learn’s PCA class uses SVD (Singular Value Decomposition) decomposition to implement PCA
pca = PCA(n_components = 2)
X2D = pca.fit_transform(X)
# The ratio indicates the proportion of the dataset’s variance that
# lies along each principal component
pca.explained_variance_ratio_

# Choose the right dimension
# The following code performs PCA without reducing dimensionality, then
# computes the minimum number of dimensions required to preserve 95%
# of the training set’s variance, You could then set n_components=d and run PCA again.
pca = PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1

# But there is a much better option: instead of specifying the number of principal
# components you want to preserve, you can set n_components to be a float
# between 0.0 and 1.0, indicating the ratio of variance you wish to preserve:
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X_train)

# Yet another option is to plot the explained variance as a function of the
# number of dimensions
plt.figure(figsize=(6,4))
plt.plot(cumsum, linewidth=3)
plt.axis([0, 400, 0, 1])
plt.xlabel("Dimensions")
plt.ylabel("Explained Variance")
plt.plot([d, d], [0, 0.95], "k:")
plt.plot([0, d], [0.95, 0.95], "k:")
plt.plot(d, 0.95, "ko")
plt.annotate("Elbow", xy=(65, 0.85), xytext=(70, 0.7),
             arrowprops=dict(arrowstyle="->"), fontsize=16)
plt.grid(True)
# save_fig("explained_variance_plot")
plt.show()

# It is also possible to decompress the reduced dataset back.
# This won’t give you back the original data, since the projection lost a bit
# of information (within the 5% variance that was dropped), but it will
# likely be close to the original data. The mean squared distance between the
# original data and the reconstructed data (compressed and then
# decompressed) is called the reconstruction error.
pca = PCA(n_components = 154)
X_reduced = pca.fit_transform(X_train)
X_recovered = pca.inverse_transform(X_reduced)

# By default, svd_solver is actually set to "auto": Scikit-Learn
# automatically uses the randomized PCA algorithm if m or n is greater than
# 500 and d is less than 80% of m or n, or else it uses the full SVD approach.
# If you want to force Scikit-Learn to use full SVD, you can set the
# svd_solver hyperparameter to "full". If you set the svd_solver hyperparameter to 
# "randomized", Scikit-Learn uses a stochastic algorithm called Randomized PCA that 
# quickly finds an approximation of the first d principal components. Its computational
# complexity is O(m × d ) + O(d ), instead of O(m × n ) + O(n ) for the full
# SVD approach, so it is dramatically faster than full SVD when d is much smaller than n.
rnd_pca = PCA(n_components=154, svd_solver="randomized")
X_reduced = rnd_pca.fit_transform(X_train)

# Incremental PCA (IPCA) algorithms allow you to split the training set into mini-batches and
# feed an IPCA algorithm one mini-batch at a time. This is useful for large
# training sets and for applying PCA online.
from sklearn.decomposition import IncrementalPCA

n_batches = 100
inc_pca = IncrementalPCA(n_components=154)
for X_batch in np.array_split(X_train, n_batches):
  inc_pca.partial_fit(X_batch)

X_reduced = inc_pca.transform(X_train)

# Kernel PCA (kPCA) 
'''
KPCA making it possible to perform complex nonlinear projections for 
dimensionality reduction. It is often good at preserving clusters
of instances after projection, or sometimes even unrolling datasets that lie
close to a twisted manifold.

As kPCA is an unsupervised learning algorithm, there is no obvious
performance measure to help you select the best kernel and
hyperparameter values. That said, dimensionality reduction is often a
preparation step for a supervised learning task (e.g., classification), so you
can use grid search to select the kernel and hyperparameters that lead to
the best performance on that task.

Another approach is to select the kernel and hyperparameters that yield the lowest reconstruction error.
'''
from sklearn.decomposition import KernelPCA

rbf_pca = KernelPCA(n_components = 2, kernel="rbf", gamma=0.04)
X_reduced = rbf_pca.fit_transform(X)

# Find the rigth kernel
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

clf = Pipeline([
                ("kpca", KernelPCA(n_components=2)),
                ("log_reg", LogisticRegression())
                ])
param_grid = [{
    "kpca__gamma": np.linspace(0.03, 0.05, 10),
    "kpca__kernel": ["rbf", "sigmoid"]
    }]

grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X, y)

In [ ]:
# LLE (Locally Linear Embedding)
'''
LLE works by first measuring how each training instance
linearly relates to its closest neighbors (c.n.), and then looking for a lowdimensional
representation of the training set where these local
relationships are best preserved (more details shortly). This approach
makes it particularly good at unrolling twisted manifolds, especially when
there is not too much noise.
'''
from sklearn.manifold import LocallyLinearEmbedding

lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10)
X_reduced = lle.fit_transform(X)

In [ ]:
# MDS
# Reduces dimensionality while trying to preserve the distances between
# the instances.
from sklearn.manifold import MDS

mds = MDS(n_components=2, random_state=42)
X_reduced_mds = mds.fit_transform(X)

In [ ]:
# Isomap
Creates a graph by connecting each instance to its nearest neighbors,
# then reduces dimensionality while trying to preserve the geodesic
# distances between the instances.
from sklearn.manifold import Isomap

isomap = Isomap(n_components=2)
X_reduced_isomap = isomap.fit_transform(X)

In [ ]:
# TSNE
# Reduces dimensionality while trying to keep similar instances close
# and dissimilar instances apart. It is mostly used for visualization, in
# particular to visualize clusters of instances in high-dimensional space.
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=42)
X_reduced_tsne = tsne.fit_transform(X)

### Models

#### **Predict Values –– Regression**

In [ ]:
'''
Based on SVD (Singular Value Decomposition), is more efficient than
the NOrmal Equation, plus it handles edge cases like not invertible 
matrix nicely. But it is not efficient (O(n2)) as the Gradient Descent
approach which is better suited for cases where there are a large number of 
features or too many training instances in fit of memory.
'''

from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

# Ridge Regression (also called Tikhonov regularization)–– a regularized version of Linear Regression
# This forces the learning algorithm to not only fit the data but 
# also keep the model weights as small as possible.
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=1, solver="cholesky", random_state=42)
ridge_reg.fit(X, y)
ridge_reg.predict([[1.5]])

# Lasso Regression (Least Absolute Shrinkage and Selection Operator Regression) 
# another regularized version of Linear Regression.
'''
An important characteristic of Lasso Regression is that it tends to eliminate
the weights of the least important features (i.e., set them to zero). In other words, 
Lasso Regression automatically performs feature selection and outputs a sparse model (i.e.,
with few nonzero feature weights).
'''
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.1)
lasso_reg.fit(X, y)
lasso_reg.predict([[1.5]])

'''
Elastic Net is a middle ground between Ridge Regression and Lasso
Regression. The regularization term is a simple mix of both Ridge and
Lasso’s regularization terms, and you can control the mix ratio r. When r =
0, Elastic Net is equivalent to Ridge Regression, and when r = 1, it is
equivalent to Lasso Regression.
'''
from sklearn.linear_model import ElasticNet
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42)
elastic_net.fit(X, y)
elastic_net.predict([[1.5]])

# How to choose from plain Linear Regression (i.e., without any regularization), Ridge, Lasso, or Elastic Net?
'''
It is almost always preferable to have at least a little bit of regularization, 
so generally you should avoid plain Linear Regression. Ridge is a good default, 
but if you suspect that only a few features are useful, you should prefer Lasso 
or Elastic Net because they tend to reduce the useless features’ weights down to 
zero, as we have discussed. In general, Elastic Net is preferred over Lasso 
because Lasso may behave erratically when the number of features is greater than the number of
training instances or when several features are strongly correlated.
'''

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)

In [ ]:
'''
The general idea of Gradient Descent is to tweak parameters iteratively 
in order to minimize a cost function. You usually start by filling θ with random 
values (this is called random initialization). When using Gradient Descent, you 
should ensure that all features have a similar scale, or else it will take much 
longer to converge.
'''

# Batch Gradient Descent

'''
The main problem with Batch Gradient Descent is the fact that it uses the
whole training set to compute the gradients at every step, which makes it
very slow when the training set is large.
'''

eta = 0.1  # learning rate
n_iterations = 1000
m = 100

theta = np.random.randn(2,1)  # random initialization for weight vector

for iteration in range(n_iterations):
    gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
    theta = theta - eta * gradients

# Visualize Gradient Descent

theta_path_bgd = []

def plot_gradient_descent(theta, eta, theta_path=None):
    m = len(X_b)
    plt.plot(X, y, "b.")
    n_iterations = 1000
    for iteration in range(n_iterations):
        if iteration < 10:
            y_predict = X_new_b.dot(theta)
            style = "b-" if iteration > 0 else "r--"
            plt.plot(X_new, y_predict, style)
        gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
        theta = theta - eta * gradients
        if theta_path is not None:
            theta_path.append(theta)
    plt.xlabel("$x_1$", fontsize=18)
    plt.axis([0, 2, 0, 15])
    plt.title(r"$\eta = {}$".format(eta), fontsize=16)

np.random.seed(42)
theta = np.random.randn(2,1)  # random initialization

plt.figure(figsize=(10,4))
plt.subplot(131); plot_gradient_descent(theta, eta=0.02)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.subplot(132); plot_gradient_descent(theta, eta=0.1, theta_path=theta_path_bgd)
plt.subplot(133); plot_gradient_descent(theta, eta=0.5)

plt.show()

# Stochastic Gradient Descent

'''
Stochastic Gradient Descent picks a random instance in the training set at every step
and computes the gradients based only on that single instance. On the other hand, 
due to its stochastic (i.e., random) nature, this algorithm is much less regular 
than Batch Gradient Descent: instead of gently decreasing until it reaches the minimum, 
the cost function will bounce up and down, decreasing only on average. Over time it 
will end up very close to the minimum, but once it gets there it will continue 
to bounce around, never settling down. So once the algorithm stops, the final
parameter values are good, but not optimal.

When the cost function is very irregular, this can actually help the algorithm 
jump out of local minima, so Stochastic Gradient Descent has a better chance of 
finding the global minimum than Batch Gradient Descent does.

The function that determines the learning rate at each iteration is called the 
learning schedule. If the learning rate is reduced too quickly, you may get stuck 
in a local minimum, or even end up frozen halfway to the minimum. If the learning 
rate is reduced too slowly, you may jump around the minimum for a long time and 
end up with a suboptimal solution if you halt training too early. 

If you want to be sure that the algorithm goes through every instance at each epoch, another
approach is to shuffle the training set (making sure to shuffle the input
features and the labels jointly), then go through it instance by instance, then
shuffle it again, and so on. However, this approach generally converges more
slowly.

When using Stochastic Gradient Descent, the training instances must be independent
and identically distributed (IID) to ensure that the parameters get pulled toward the
global optimum, on average. A simple way to ensure this is to shuffle the instances
during training (e.g., pick each instance randomly, or shuffle the training set at the
beginning of each epoch). If you do not shuffle the instances—for example, if the
instances are sorted by label—then SGD will start by optimizing for one label, then the
next, and so on, and it will not settle close to the global minimum.
'''
# Use library
from sklearn.linear_model import SGDRegressor

sgd_reg = SGDRegressor(max_iter=1000, tol=1e-3, penalty=None, eta0=0.1, random_state=42)
sgd_reg.fit(X, y.ravel())

# Under the hood
theta_path_sgd = []
m = len(X_b)
np.random.seed(42)

n_epochs = 50
t0, t1 = 5, 50  # learning schedule hyperparameters

def learning_schedule(t):
    return t0 / (t + t1)

theta = np.random.randn(2,1)  # random initialization

for epoch in range(n_epochs):
    for i in range(m):
        if epoch == 0 and i < 20:          # only display the first 20 steps        
            y_predict = X_new_b.dot(theta)           
            style = "b-" if i > 0 else "r--"       
            plt.plot(X_new, y_predict, style)      
        random_index = np.random.randint(m)
        xi = X_b[random_index:random_index+1]
        yi = y[random_index:random_index+1]
        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(epoch * m + i)
        theta = theta - eta * gradients
        theta_path_sgd.append(theta)              

plt.plot(X, y, "b.")                              
plt.xlabel("$x_1$", fontsize=18)                    
plt.ylabel("$y$", rotation=0, fontsize=18)          
plt.axis([0, 2, 0, 15])                              
plt.show()                                          

# Mini-batch Gradient Descent
'''
At each step, Mini-batch GD computes the gradients on small random sets of 
instances called mini-batches. The main advantage of Minibatch
GD over Stochastic GD is that you can get a performance boost from
hardware optimization of matrix operations, especially when using GPUs.

The algorithm’s progress in parameter space is less erratic than with
Stochastic GD, especially with fairly large mini-batches. As a result, Minibatch
GD will end up walking around a bit closer to the minimum than
Stochastic GD—but it may be harder for it to escape from local minima (in
the case of problems that suffer from local minima, unlike Linear
Regression).
'''

theta_path_mgd = []

n_iterations = 50
minibatch_size = 20

np.random.seed(42)
theta = np.random.randn(2,1)  # random initialization

t0, t1 = 200, 1000
def learning_schedule(t):
    return t0 / (t + t1)

t = 0
for epoch in range(n_iterations):
    shuffled_indices = np.random.permutation(m)
    X_b_shuffled = X_b[shuffled_indices]
    y_shuffled = y[shuffled_indices]
    for i in range(0, m, minibatch_size):
        t += 1
        xi = X_b_shuffled[i:i+minibatch_size]
        yi = y_shuffled[i:i+minibatch_size]
        gradients = 2/minibatch_size * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(t)
        theta = theta - eta * gradients
        theta_path_mgd.append(theta)

# Perform Ridge Regression using Stochastic Gradient Descent.
# The penalty hyperparameter sets the type of regularization term to use.
# Specifying "l2" indicates that you want SGD to add a regularization term to
# the cost function equal to half the square of the ℓ norm of the weight vector:
# this is simply Ridge Regression. "l1" means Lasso Regression.
sgd_reg = SGDRegressor(penalty="l2", max_iter=1000, tol=1e-3, random_state=42)
sgd_reg.fit(X, y.ravel())
sgd_reg.predict([[1.5]])

# Early stopping example
'''
With Stochastic and Mini-batch Gradient Descent, the curves are not so smooth, and it
may be hard to know whether you have reached the minimum or not. One solution is to
stop only after the validation error has been above the minimum for some time (when
you are confident that the model will not do any better), then roll back the model
parameters to the point where the validation error was at a minimum.
'''
from copy import deepcopy

poly_scaler = Pipeline([
        ("poly_features", PolynomialFeatures(degree=90, include_bias=False)),
        ("std_scaler", StandardScaler())
    ])

X_train_poly_scaled = poly_scaler.fit_transform(X_train)
X_val_poly_scaled = poly_scaler.transform(X_val)

sgd_reg = SGDRegressor(max_iter=1, tol=-np.infty, warm_start=True,
                       penalty=None, learning_rate="constant", eta0=0.0005, random_state=42)

minimum_val_error = float("inf")
best_epoch = None
best_model = None
for epoch in range(1000):
    sgd_reg.fit(X_train_poly_scaled, y_train)  # continues where it left off
    y_val_predict = sgd_reg.predict(X_val_poly_scaled)
    val_error = mean_squared_error(y_val, y_val_predict)
    if val_error < minimum_val_error:
        minimum_val_error = val_error
        best_epoch = epoch
        best_model = deepcopy(sgd_reg)


In [ ]:
# Polynomial Regression

from sklearn.preprocessing import PolynomialFeatures
# PolynomialFeatures also adds all combinations of features up to the given degree.
# Like a3, b3, a2b, ab2
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly_features.fit_transform(X) # Expand features

lin_reg = LinearRegression() # Use linear regression models to do the polynomial
lin_reg.fit(X_poly, y)
lin_reg.intercept_, lin_reg.coef_

In [ ]:
# Support vector mahchine (SVM) regression
'''
To use SVMs for regression instead of classification, the trick is to reverse the objective: 
instead of trying to fit the largest possible street between two classes while limiting margin
violations, SVM Regression tries to fit as many instances as possible on the street while
limiting margin violations (i.e., instances off the street). The width of the street is
controlled by a hyperparameter, ϵ (called tol in Scikit-Learn, it's like the margin width).

The SVR class is the regression equivalent of the SVC class, and the LinearSVR class is the
regression equivalent of the LinearSVC class. The LinearSVR class scales linearly with the
size of the training set (just like the LinearSVC class), while the SVR class gets much too
slow when the training set grows large (just like the SVC class).
'''

from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=1.5, random_state=42)
svm_reg.fit(X, y)

# Kernelized support vector mahchine (SVM) for nonlinear regression

from sklearn.svm import SVR

svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1, gamma="scale")
svm_poly_reg.fit(X, y)

In [ ]:
# Decision tree regression

from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(random_state=42, max_depth=2)
tree_reg1.fit(X, y)

#### **Predict Class –– Classification**

In [ ]:
# SGD Classifier
'''
This classifier has the advantage of being capable of handling very large 
datasets efficiently. This is in part because SGD deals with training instances 
independently, one at a time (which also makes SGD well suited for online learning).
Capable of handling multiple classes.
'''
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=168)
sgd_clf.fit(X_train, y_train_5)
sgd_clf.predict([X[0]])

In [ ]:
# Decision tree
'''
Decision Trees are versatile Machine Learning algorithms that can perform both
classification and regression tasks, and even multioutput tasks.

Decision Trees require very little data preparation. In fact, they
don’t require feature scaling or centering at all.

Scikit-Learn uses the CART (Classification and Regression Tree) algorithm, which 
produces only binary trees: nonleaf nodes always have two children (i.e., questions 
only have yes/no answers). However, other algorithms such as ID3 can produce
Decision Trees with nodes that have more than two children.

Decision tree has the predict_proba() method.

Most of the time entropy and Gini do not make a big difference: they lead to 
similar trees. Gini impurity is slightly faster to compute, so it is a good default. 
However, when they differ, Gini impurity tends to isolate the most frequent class 
in its own branch of the tree, while entropy tends to produce slightly more balanced trees.

Increasing min_* hyperparameters or reducing max_* hyperparameters will regularize the model.

Decision Trees have some limitations: Decision Trees love orthogonal decision boundaries 
(all splits are perpendicular to an axis), which makes them sensitive to training set rotation, 
one way to limit this problem is to use Principal Component Analysis;
the main issue with Decision Trees is that they are very sensitive to small
variations in the training data (Actually, since the training algorithm used by 
Scikit-Learn is stochastic, you may get very different models even on the
same training data (unless you set the random_state hyperparameter)), 
random Forests can limit this instability by averaging predictions over many trees.
'''

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:, 2:] # petal length and width
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2, random_state=42)
tree_clf.fit(X, y)

# Visualize decision tree

from graphviz import Source
from sklearn.tree import export_graphviz

export_graphviz(
        tree_clf,
        out_file=os.path.join(IMAGES_PATH, "iris_tree.dot"),
        feature_names=iris.feature_names[2:],
        class_names=iris.target_names,
        rounded=True,
        filled=True
    )

# convert .dot to .png
# $ dot -Tpng iris_tree.dot -o iris_tree.png

Source.from_file(os.path.join(IMAGES_PATH, "iris_tree.dot"))

# I don't want to save these plots, so just show them
def draw_tree_graph(tree):
    data = export_graphviz(tree, 
                           feature_names=iris.feature_names, class_names=iris.target_names, 
                           filled=True, rounded=True
                          )
    graph = Source(data)
    return graph

In [ ]:
from sklearn.svm import SVC

svm_clf = SVC()
svm_clf.fit(X_train, y_train)

In [ ]:
# KNN 
# can handle multilabel, and multioutput

from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_multilabel)

In [ ]:
# Logistic Regression
'''
The bad news is that there is no known closed-form equation to compute the
value of θ that minimizes this cost function (there is no equivalent of the
Normal Equation). The good news is that this cost function is convex, so
Gradient Descent (or any other optimization algorithm) is guaranteed to find
the global minimum (if the learning rate is not too large and you wait long
enough).

Just like the other linear models, Logistic Regression models can be
regularized using ℓ or ℓ penalties. Scikit-Learn actually adds an ℓ penalty
by default.

The hyperparameter controlling the regularization strength of a Scikit-Learn
LogisticRegression model is not alpha (as in other linear models), but its inverse:
C. The higher the value of C, the less the model is regularized.
'''

from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(solver="lbfgs", random_state=42)
log_reg.fit(X, y)

In [ ]:
# Softmax Regression / Multinomial Logiistic Regression
'''
The Logistic Regression model can be generalized to support multiple
classes directly, without having to train and combine multiple binary
classifiers.

The idea is simple: when given an instance x, the Softmax Regression model
first computes a score s (x) for each class k, then estimates the probability of
each class by applying the softmax function (also called the normalized exponential) 
to the scores.

Just like the Logistic Regression classifier, the Softmax Regression
classifier predicts the class with the highest estimated probability (which is
simply the class with the highest score),

The Softmax Regression classifier predicts only one class at a time (i.e., it is multiclass,
not multioutput), so it should be used only with mutually exclusive classes, such as
different types of plants. You cannot use it to recognize multiple people in one picture.

Scikit-Learn’s LogisticRegression uses one-versus-the-rest by
default when you train it on more than two classes, but you can set the
multi_class hyperparameter to "multinomial" to switch it to Softmax
Regression. You must also specify a solver that supports Softmax
Regression, such as the "lbfgs" solver (see Scikit-Learn’s documentation
for more details). It also applies ℓ regularization by default, which you can
control using the hyperparameter C.
'''

softmax_reg = LogisticRegression(multi_class="multinomial", solver="lbfgs", C=10, random_state=42)
softmax_reg.fit(X, y)
softmax_reg.predict([[5, 2]])
softmax_reg.predict_proba([[5, 2]])

In [ ]:
# Support vector machine (SVM)
'''
Support vectors are the instances located on the edge.

SVMs are sensitive to the feature scales, for example, when the vertical scale is
much larger than the horizontal scale, the widest possible street is close to horizontal.

It's a binary classifier.
'''
# linear SVM model (using the LinearSVC class with C=1 and the hinge loss function) for linear classification
'''
The LinearSVC class regularizes the bias term, so you should center the training set first by subtracting its
mean. This is automatic if you scale the data using the StandardScaler. Also make sure you set the loss
hyperparameter to "hinge", as it is not the default value. Finally, for better performance, you should set
the dual hyperparameter to False, unless there are more features than training instances.

As a rule of thumb, you should always try the linear kernel first (remember that LinearSVC is much faster than
SVC(kernel="linear"). The LinearSVC class is based on the liblinear library, which implements an optimized
algorithm for linear SVMs. It does not support the kernel trick, but it scales almost
linearly with the number of training instances and the number of features. Its training time
complexity is roughly O(m × n)), especially if the training set is very large or if it has plenty of features. If the
training set is not too large, you should also try the Gaussian RBF kernel; it works well in most cases.

Then if you have spare time and computing power, you can experiment with a few other kernels, using
cross-validation and grid search. You’d want to experiment like that especially if there are kernels
specialized for your training set’s data structure.
'''
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris["data"][:, (2, 3)]  # petal length, petal width
y = (iris["target"] == 2).astype(np.float64)  # Iris virginica

svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("linear_svc", LinearSVC(C=1, loss="hinge", random_state=42)),
    ])

svm_clf.fit(X, y)

# Instead of using the LinearSVC class, we could use the SVC class with a linear kernel.
'''
Or we could use the SGDClassifier class, with SGDClassifier(loss="hinge",
alpha=1/(m*C)). This applies regular Stochastic Gradient Descent to train
a linear SVM classifier. It does not converge as fast as the LinearSVC class, but it can be
useful to handle online classification tasks or huge datasets that do not fit in memory (outof-
core training).
'''
from sklearn.svm import SVC
svm_clf = SVC(kernel="linear", C=1)
svm_clf.fit(X, y)

# linear SVM model for non-linear classification by adding polynomial features
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

X, y = make_moons(n_samples=100, noise=0.15) 
polynomial_svm_clf = Pipeline([
        ("poly_features", PolynomialFeatures(degree=3)), 
        ("scaler", StandardScaler()),
        ("svm_clf", LinearSVC(C=10, loss="hinge", random_state=42))
    ])

polynomial_svm_clf.fit(X, y)

# linear SVM model for non-linear classification by poly kernel trick
from sklearn.svm import SVC

poly_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
#       The hyperparameter coef0 controls how much the model is influenced by high-degree
#       polynomials versus low-degree polynomials.
    ])
poly_kernel_svm_clf.fit(X, y)

# linear SVM model for non-linear classification by similar function - Gaussian RBF
'''
Just like the polynomial features method, the similarity features method can be useful with
any Machine Learning algorithm, but it may be computationally expensive to compute all
the additional features, especially on large training sets. So we need kernel trick.

Increasing gamma makes the bell-shaped curve narrower. As a result, each
instance’s range of influence is smaller: the decision boundary ends up being more
irregular, wiggling around individual instances. Conversely, a small gamma value makes the
bell-shaped curve wider: instances have a larger range of influence, and the decision
boundary ends up smoother. So γ acts like a regularization hyperparameter: if your model
is overfitting, you should reduce it; if it is underfitting, you should increase it (similar to
the C hyperparameter).
'''
rbf_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
    ])
rbf_kernel_svm_clf.fit(X, y)


#### **Clustering**

In [ ]:
# KMeans
# Unsupervised model
from sklearn.cluster import KMeans

k = 5
kmeans = KMeans(n_clusters=k)
y_pred = kmeans.fit_predict(X)

# Mini Batch KMeans
# Faster and suit for out-of-core sataset
from sklearn.cluster import MiniBatchKMeans
minibatch_kmeans = MiniBatchKMeans(n_clusters=5)
minibatch_kmeans.fit(X)

# Find the optimal number of clusters
kmeans_per_k = [KMeans(n_clusters=k, random_state=42).fit(X)
                for k in range(1, 10)]
inertias = [model.inertia_ for model in kmeans_per_k]

plt.figure(figsize=(8, 3.5))
plt.plot(range(1, 10), inertias, "bo-")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Inertia", fontsize=14)
plt.annotate('Elbow',
             xy=(4, inertias[3]),
             xytext=(0.55, 0.55),
             textcoords='figure fraction',
             fontsize=16,
             arrowprops=dict(facecolor='black', shrink=0.1)
            )
plt.axis([1, 8.5, 0, 1300])
# save_fig("inertia_vs_k_plot")
plt.show()

from sklearn.metrics import silhouette_score

silhouette_scores = [silhouette_score(X, model.labels_)
                     for model in kmeans_per_k[1:]]

plt.figure(figsize=(8, 3))
plt.plot(range(2, 10), silhouette_scores, "bo-")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Silhouette score", fontsize=14)
plt.axis([1.8, 8.5, 0.55, 0.7])
# save_fig("silhouette_score_vs_k_plot")
plt.show()

### Ensamble

In [ ]:
# Voting classifiers

'''
Ensemble methods work best when the predictors are as independent from one another as possible.
One way to get diverse classifiers is to train them using very different algorithms. This increases the
chance that they will make very different types of errors, improving the ensemble’s accuracy.

A very simple way to create an even better classifier is to aggregate the predictions of
each classifier and predict the class that gets the most votes. This majority-vote
classifier is called a hard voting classifier.

If all classifiers are able to estimate class probabilities (i.e., they all have a
predict_proba() method), then you can tell Scikit-Learn to predict the class with the
highest class probability, averaged over all the individual classifiers. This is called soft
voting. It often achieves higher performance than hard voting because it gives more
weight to highly confident votes. All you need to do is replace voting="hard" with
voting="soft" and ensure that all classifiers can estimate class probabilities.
'''
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", random_state=42)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')

voting_clf.fit(X_train, y_train)

# soft voting
log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", probability=True, random_state=42) # notice the probability here

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='soft')
voting_clf.fit(X_train, y_train)

In [ ]:
# Bagging and pasting
'''
Use the same training algorithm for every
predictor and train them on different random subsets of the training set. When sampling
is performed with replacement, this method is called bagging (short for bootstrap
aggregating ). When sampling is performed without replacement, it is called pasting.
In other words, both bagging and pasting allow training instances to be sampled several
times across multiple predictors, but only bagging allows training instances to be
sampled several times for the same predictor.

Predictors can all be trained in parallel, via different CPU
cores or even different servers. Similarly, predictions can be made in parallel. This is
one of the reasons bagging and pasting are such popular methods: they scale very well.

Bootstrapping introduces a bit more diversity in the subsets that each predictor is
trained on, so bagging ends up with a slightly higher bias than pasting; but the extra
diversity also means that the predictors end up being less correlated, so the ensemble’s
variance is reduced. Overall, bagging often results in better models, which explains why
it is generally preferred. However, if you have spare time and CPU power, you can use
cross-validation to evaluate both bagging and pasting and select the one that works best.
'''
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=42), n_estimators=500,
    max_samples=100, bootstrap=True, random_state=42)
# if you want to use pasting instead, just set bootstrap=False).
# The n_jobs parameter tells Scikit-Learn the number of CPU cores to use for training 
# and predictions (–1 tells Scikit-Learn to use all available cores).
# The BaggingClassifier automatically performs soft voting instead of hard voting if the base
# classifier can estimate class probabilities.
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

# In Scikit-Learn, you can set oob_score=True when creating a BaggingClassifier to
# request an automatic oob evaluation after training. By default a BaggingClassifier samples m
# training instances with replacement (bootstrap=True), where m is the size of the
# training set. This means that only about 63% of the training instances are sampled on
# average for each predictor. The remaining 37% of the training instances that are not
# sampled are called out-of-bag (oob) instances. Note that they are not the same 37% for
# all predictors. Since a predictor never sees the oob instances during training, 
# it can be evaluated on these instances, without the need for a separate validation set.
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=42), n_estimators=500,
    bootstrap=True, oob_score=True, random_state=40)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

# The BaggingClassifier class supports sampling the features as well. Sampling is
# controlled by two hyperparameters: max_features and bootstrap_features. 
# This technique is particularly useful when you are dealing with high-dimensional inputs
# (such as images). Sampling both training instances and features is called the Random
# Patches method. Keeping all training instances (by setting bootstrap=False and
# max_samples=1.0) but sampling features (by setting bootstrap_features to True
# and/or max_features to a value smaller than 1.0) is called the Random Subspaces
# method. Sampling features results in even more predictor diversity, trading a 
# bit more bias for a lower variance.

In [ ]:
# Random forest
# Capable of handling multiple classes.
'''
A Random Forest is an ensemble of Decision Trees, generally trained via the bagging 
method (or sometimes pasting), typically with max_samples set to the size of the 
training set (similarly, there is a RandomForestRegressor class for regression tasks).

The Random Forest algorithm introduces extra randomness when growing trees; instead
of searching for the very best feature when splitting a node, it searches
for the best feature among a random subset of features. The algorithm results in greater
tree diversity, which (again) trades a higher bias for a lower variance, generally yielding
an overall better model.
'''
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)
y_pred_rf = rnd_clf.predict(X_test)

# Scikit-Learn measures a feature’s importance by
# looking at how much the tree nodes that use that feature reduce impurity on average
# (across all trees in the forest).
rnd_clf.feature_importances_

In [ ]:
# Boosting
'''
Boosting (originally called hypothesis boosting) refers to any Ensemble method that can
combine several weak learners into a strong learner. The general idea of most boosting
methods is to train predictors sequentially, each trying to correct its predecessor.
'''

# AdaBoost
'''
One way for a new predictor to correct its predecessor is to pay a bit more attention to
the training instances that the predecessor underfitted. This results in new predictors
focusing more and more on the hard cases. This is the technique used by AdaBoost.

There is one important drawback to this sequential learning technique: it cannot be parallelized (or
only partially), since each predictor can only be trained after the previous predictor has been trained
and evaluated. As a result, it does not scale as well as bagging or pasting.

Scikit-Learn uses a multiclass version of AdaBoost called SAMME (which stands for
Stagewise Additive Modeling using a Multiclass Exponential loss function). When there
are just two classes, SAMME is equivalent to AdaBoost. If the predictors can estimate
class probabilities (i.e., if they have a predict_proba() method), Scikit-Learn can use
a variant of SAMME called SAMME.R (the R stands for “Real”), which relies on class
probabilities rather than predictions and generally performs better.

If your AdaBoost ensemble is overfitting the training set, you can try reducing the number of
estimators or more strongly regularizing the base estimator.
'''

from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5, random_state=42)
ada_clf.fit(X_train, y_train)

# GRadient boosting
'''
Gradient Boosting works by sequentially adding predictors to an ensemble, each one
correcting its predecessor. However, instead of tweaking the instance weights at every
iteration like AdaBoost does, this method tries to fit the new predictor to the residual
errors made by the previous predictor.

The GradientBoostingRegressor class also supports a subsample hyperparameter,
which specifies the fraction of training instances to be used for training each tree. For
example, if subsample=0.25, then each tree is trained on 25% of the training instances,
selected randomly. This technique trades a higher bias for a lower variance. It also 
speeds up training considerably. This is called Stochastic Gradient Boosting.
'''

from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0, random_state=42)
gbrt.fit(X, y)

# Gradient boost with early stopping 1 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=49)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120, random_state=42)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred)
          for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors) + 1

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators, random_state=42)
gbrt_best.fit(X_train, y_train)

# Gradient boost with early stopping 2
# The following code stops training when the validation error does not improve for five iterations in a row:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True, random_state=42)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break  # early stopping

# It is worth noting that an optimized implementation of Gradient Boosting is available in
# the popular Python library XGBoost, which stands for Extreme Gradient Boosting. This
# package was initially developed by Tianqi Chen as part of the Distributed (Deep)
# Machine Learning Community (DMLC), and it aims to be extremely fast, scalable, and
# portable. In fact, XGBoost is often an important component of the winning entries in
# ML competitions. XGBoost’s API is quite similar to Scikit-Learn’s:
import xgboost

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)
# XGBoost also offers several nice features, such as automatically taking care of early
# stopping:
xgb_reg.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=2)
y_pred = xgb_reg.predict(X_val)


# Under the hood
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg1.fit(X, y)

y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg2.fit(X, y2)

y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg3.fit(X, y3)

y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [ ]:
# Stacking

### Evaluate 

**Regression**

In [ ]:
# mean squared error for regression

from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

In [ ]:
# mean squared error for regression with cross validation
'''
Scikit-Learn’s cross-validation features expect a utility function (greater is better) 
rather than a cost function (lower is better), so the scoring function is actually 
the opposite of the MSE (i.e., a negative value), which is why the following code 
computes -scores before calculating the square root.

If GridSearchCV is initialized with refit=True (which is the default), then once it finds
the best estimator using cross-validation, it retrains it on the whole training set. This is
usually a good idea, since feeding it more data will likely improve its performance.

If a model performs well on the training data but generalizes poorly according 
to the cross-validation metrics, then your model is overfitting. If it performs 
poorly on both, then it is underfitting.
'''
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)
tree_rmse_scores

In [ ]:
# 95% confidence interval for the generalization error
from scipy import stats

confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                         loc=squared_errors.mean(),
                         scale=stats.sem(squared_errors)))

# compute the interval manually
m = len(squared_errors)
mean = squared_errors.mean()
tscore = stats.t.ppf((1 + confidence) / 2, df=m - 1)
tmargin = tscore * squared_errors.std(ddof=1) / np.sqrt(m)
np.sqrt(mean - tmargin), np.sqrt(mean + tmargin)

In [ ]:
# Check overfitting or underfitting by plot learning curves
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

'''
Check the error first.
If there is a gap between the curves. This means that the model
performs significantly better on the training data than on the
validation data, which is the hallmark of an overfitting model. If
you used a much larger training set, however, the two curves would
continue to get closer.
'''
def plot_learning_curves(model, X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=10)
    train_errors, val_errors = [], []
    for m in range(1, len(X_train)):
        model.fit(X_train[:m], y_train[:m])
        y_train_predict = model.predict(X_train[:m])
        y_val_predict = model.predict(X_val)
        train_errors.append(mean_squared_error(y_train[:m], y_train_predict))
        val_errors.append(mean_squared_error(y_val, y_val_predict))

    plt.plot(np.sqrt(train_errors), "r-+", linewidth=2, label="train")
    plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="val")
    plt.legend(loc="upper right", fontsize=14)   # not shown in the book
    plt.xlabel("Training set size", fontsize=14) # not shown
    plt.ylabel("RMSE", fontsize=14)              # not shown

**Classification**

In [ ]:
# implement cross validation by stratified k fold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

skfold = StratifiedKFold(n_splits=3, random_state=168)

for train_index, test_index in skfold.split(X_train, y_train_5):
  clone_sgd_clf = clone(sgd_clf)
  X_train_folds = X_train[train_index]
  y_train_folds = y_train_5[train_index]
  X_test_folds = X_train[test_index]
  y_test_folds = y_train_5[test_index]

  clone_sgd_clf.fit(X_train_folds, y_train_folds)
  y_pred = clone_sgd_clf.predict(X_test_folds)
  n_correct = sum(y_pred == y_test_folds)
  print(f'Correct rate is: {n_correct / len(y_pred)}')

'''
accuracy is generally not the preferred performance measure for classifiers, 
especially when you are dealing with skewed datasets (i.e., when some classes 
are much more frequent than others)
'''

In [ ]:
# confusion matrics
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)

from sklearn.metrics import confusion_matrix

confusion_matrix(y_train_5, y_train_pred)
# true negatives   |  false positives
# false negatives  |  true positives
# precision = TP / (TP + FP)
# recall (sensitivity, true positive rate (TPR)) = TP / (TP + FN)  the ratio of positive instances that are correctly detected by the classifier
'''
F1 score = 2 * (precision * recall) / (precision + recall) 
F1 is the harmonic mean of precision and recall. Whereas the regular mean treats 
all values equally, the harmonic mean gives much more weight to low values. 
As a result, the classifier will only get a high F1 score if both recall and precision are high.
'''
# precision/recall trade-off
from sklearn.metrics import precision_score, recall_score, f1_score

print(f'precision: {precision_score(y_train_5, y_train_pred)}, recall: {recall_score(y_train_5, y_train_pred)}, f1: {f1_score(y_train_5, y_train_pred)}')

In [ ]:
# PR curve (Precision and recall versus the decision threshold)
# help choose the threshold for binary classifier (precision-recall trade-off)
'''
As a rule of thumb, you should prefer the PR curve
whenever the positive class is rare or when you care more about the false positives
than the false negatives. Otherwise, use the ROC curve.
'''

y_scores = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3, method='decision_function')

from sklearn.metrics import  precision_recall_curve

precisions, recalls, thresholds = precision_recall_curve(y_train_5,
y_scores)

def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
    plt.legend(loc="center right", fontsize=16) 
    plt.xlabel("Threshold", fontsize=16)        
    plt.grid(True)                              
    plt.axis([-50000, 50000, 0, 1])            

recall_90_precision = recalls[np.argmax(precisions >= 0.90)] # get the index of recalls when precision = 0.90
# In case of multiple occurrences of the maximum values, the indices corresponding to the first occurrence are returned.
# argmax returns the first index when precision >= 0.90 is true 
# and the precisions is sorted by the threshold
threshold_90_precision = thresholds[np.argmax(precisions >= 0.90)]

plt.figure(figsize=(8, 4))                                                                  
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.plot([threshold_90_precision, threshold_90_precision], [0., 0.9], "r:")  # vertical line               
plt.plot([-50000, threshold_90_precision], [0.9, 0.9], "r:") # horizontal line 1                               
plt.plot([-50000, threshold_90_precision], [recall_90_precision, recall_90_precision], "r:") # horizontal liine 2
plt.plot([threshold_90_precision], [0.9], "ro") # point 1                                            
plt.plot([threshold_90_precision], [recall_90_precision], "ro") # point 2                            
plt.show()

In [ ]:
# ROC curve and are under the curve(AUC)
'''
The receiver operating characteristic (ROC) curve is another common
tool used with binary classifiers. It plots the true positive rate (another name 
for recall) against the false positive rate (FPR). The FPR is the ratio of 
negative instances that are incorrectly classified as positive. 
It is equal to 1 – the true negative rate (TNR), which is the ratio of negative 
instances that are correctly classified as negative.
The TNR is also called specificity. Hence, the ROC curve plots sensitivity
(recall) versus 1 – specificity.
The higher the recall (TPR), the more false positives (FPR) the classifier produces.
'''

from sklearn.model_selection import cross_val_predict

y_scores = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3, method='decision_function') # a score calculated by the classifier

from sklearn.metrics import  precision_recall_curve

precisions, recalls, thresholds = precision_recall_curve(y_train_5, y_scores)

from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_train_5, y_scores)

def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--') # dashed diagonal represents the ROC curve of a purely random classifier
    plt.axis([0, 1, 0, 1])                                    
    plt.xlabel('False Positive Rate (Fall-Out)', fontsize=16)
    plt.ylabel('True Positive Rate (Recall)', fontsize=16)    
    plt.grid(True)                                           

plt.figure(figsize=(8, 6))                                   
plot_roc_curve(fpr, tpr)
recall_90_precision = recalls[np.argmax(precisions >= 0.90)] 
fpr_90 = fpr[np.argmax(tpr >= recall_90_precision)]        
plt.plot([fpr_90, fpr_90], [0., recall_90_precision], "r:")  
plt.plot([0.0, fpr_90], [recall_90_precision, recall_90_precision], "r:") 
plt.plot([fpr_90], [recall_90_precision], "ro")                                              
plt.show()

from sklearn.metrics import roc_auc_score
'''
A perfect classifier will have a ROC AUC equal to 1, whereas a
purely random classifier will have a ROC AUC equal to 0.5.
'''
roc_auc_score(y_train_5, y_scores)

In [ ]:
# ROC and AUC by probability
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(random_state=168)
y_probas_forest = cross_val_predict(forest_clf, X_train, y_train_5, cv=3, method='predict_proba')

y_scores_forest = y_probas_forest[:, 1] # score = proba of positive class
fpr_forest, tpr_forest, thresholds_forest = roc_curve(y_train_5,y_scores_forest)

recall_90_precision = recalls[np.argmax(precisions >= 0.90)] 
fpr_90 = fpr[np.argmax(tpr >= recall_90_precision)]    
recall_for_forest = tpr_forest[np.argmax(fpr_forest >= fpr_90)]

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, "b:", linewidth=2, label="SGD")
plot_roc_curve(fpr_forest, tpr_forest, "Random Forest")
plt.plot([fpr_90, fpr_90], [0., recall_90_precision], "r:")
plt.plot([0.0, fpr_90], [recall_90_precision, recall_90_precision], "r:")
plt.plot([fpr_90], [recall_90_precision], "ro")
plt.plot([fpr_90, fpr_90], [0., recall_for_forest], "r:")
plt.plot([fpr_90], [recall_for_forest], "ro")
plt.grid(True)
plt.legend(loc="lower right", fontsize=16)
plt.show()

### Fine-Tune the Model

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
              {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]}, 
              {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
              ]
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
# grid_search = GridSearchCV(knn_clf, param_grid, cv=5, verbose=3)
grid_search.fit(housing_prepared, housing_labels)

grid_search.best_params_
grid_search.best_estimator_
pd.DataFrame(grid_search.cv_results_)
cvres = grid_search.cv_results_

for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
  print(np.sqrt(-mean_score), params)

feature_importances = grid_search.best_estimator_.feature_importances_
sorted(zip(feature_importances, attributes), reverse=True)

In [ ]:
# similar as GridSearchCV but only need to set the iteration parameter
from sklearn.model_selection import RandomizedSearchCV


### Save and Load Model

In [ ]:
full_pipeline_with_predictor = Pipeline([
        ("preparation", full_pipeline),
        ("linear", LinearRegression())
    ])

full_pipeline_with_predictor.fit(housing, housing_labels)
full_pipeline_with_predictor.predict(some_data)

my_model = full_pipeline_with_predictor

In [ ]:
import joblib
joblib.dump(my_model, "my_model.pkl")
# and later...
my_model_loaded = joblib.load("my_model.pkl")

# Deep Learning

### Tools

In [ ]:
#convert sparse labels (i.e., class indices) to one-hot vector labels
keras.utils.to_categorical() 

In [ ]:
# plot model
keras.utils.plot_model(model, "my_fashion_mnist_model.png", show_shapes=True)

In [ ]:
# visualize history
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

In [ ]:
# Save and reload model
'''
HDF5 format saves both the model’s architecture
(including every layer’s hyperparameters) and the values of all the model
parameters for every layer (e.g., connection weights and biases). It also
saves the optimizer (including its hyperparameters and any state it may
have).
'''
model.save("my_keras_model.h5")
model = keras.models.load_model("my_keras_model.h5")

# using callbacks

# checkpoint callback
'''
The ModelCheckpoint callback saves checkpoints of your model at regular intervals 
during training, by default at the end of each epoch.

If you use a validation set during training, you can set
save_best_only=True when creating the ModelCheckpoint. In this case,
it will only save your model when its performance on the validation set is
the best so far.
'''

keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])  

model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb])

model = keras.models.load_model("my_keras_model.h5") # rollback to best model
mse_test = model.evaluate(X_test, y_test)

# Combine Checkpoint and EarlyStopping callback
'''
It will interrupt training when it measures no
progress on the validation set for a number of epochs (defined by the
patience argument), and it will optionally roll back to the best model.

You can combine both callbacks to save checkpoints of your model (in
case your computer crashes) and interrupt training early when there is no
more progress (to avoid wasting time and resources), so no need to use save_best_only.
'''
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, early_stopping_cb])
mse_test = model.evaluate(X_test, y_test)


### Models

#### Classification

In [ ]:
# Perceptron
'''
Scikit-Learn’s Perceptron class is equivalent to using an SGDClassifier with the following
hyperparameters: loss="perceptron", learning_rate="constant", eta0=1 (the learning rate), 
and penalty=None (no regularization).

Contrary to Logistic Regression classifiers, Perceptrons do not
output a class probability; rather, they make predictions based on a hard
threshold. This is one reason to prefer Logistic Regression over
Perceptrons.
'''

import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

iris = load_iris()
X = iris.data[:, (2, 3)]  # petal length, petal width
y = (iris.target == 0).astype(np.int)

per_clf = Perceptron(max_iter=1000, tol=1e-3, random_state=42)
per_clf.fit(X, y)

y_pred = per_clf.predict([[2, 0.5]])

In [ ]:
# Classification MLP with two hidden layers

from tensorflow.keras import layers
model = keras.models.Sequential()
# Flatten layer convert each input image into a 1D array
model.add(keras.layers.Flatten(input_shape=[28, 28]))
# model.add(keras.layers.InputLayer(input_shape=(28, 28))) can use this layer to input, too
model.add(keras.layers.Dense(300, activation="relu"))
# model.add(keras.layers.Dense(300, activation=keras.activations.relu)) former line is same as this line
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

# another way to write the former model
model = keras.models.Sequential([
                                 layers.Flatten(input_shape=[28, 28]),
                                 layers.Dense(300, activation="relu"), 
                                 layers.Dense(100, activation="relu"),
                                 layers.Dense(10, activation="softmax")
                                 ])

model.summary()
weights, biases = model.layers[1].get_weights()

history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
# history = model.fit(X_train, y_train, epochs=10, validation_split=0.1) use the last 10% of the data (before shuffling) for validation.
'''
If the training set was very skewed, set the class_weight argument when calling the fit() method, which would
give a larger weight to underrepresented classes and a lower weight to overrepresented classes. 
These weights would be used by Keras when computing the loss.

If you need per-instance weights, set the sample_weight argument (it supersedes class_weight). 
Per-instance weights could be useful if some instances were labeled by experts while
others were labeled using a crowdsourcing platform.

It’s as simple as calling the fit() method again, since Keras just continues training where it left off.
'''
model.evaluate(X_test, y_test)
y_pred_proba = model.predict(X_test[:3])
y_pred_proba.round(2)

y_pred_index = np.argmax(model.predict(X_test[:3]), axis=1)
y_pred_class = np.array(class_names)[y_pred_index]
# Fine-tune
'''
The first one to check is the learning
rate. If that doesn’t help, try another optimizer (and always retune the
learning rate after changing any hyperparameter). If the performance is
still not great, then try tuning model hyperparameters such as the number
of layers, the number of neurons per layer, and the types of activation
functions to use for each hidden layer. You can also try tuning other
hyperparameters, such as the batch size (it can be set in the fit() method
using the batch_size argument, which defaults to 32).
'''



#### Regression

In [ ]:
# regression MLP using Sequential API
model = keras.models.Sequential([
                                 keras.layers.Dense(30, activation='relu', input_shape=X_train.shape[1:]),
                                 keras.layers.Dense(1)
])

model.compile(loss='mean_squared_error', optimizer='sgd')
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

mse_test = model.evaluate(X_test, y_test)
X_samples = X_test[:3]
y_pred = model.predict(X_samples)

In [ ]:
# regression MLP using Functional API

input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation='relu')(input_)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)
concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output])

In [ ]:
# Wide & Deep neural network
# regression MLP using Functional API
# with multiple inputs from wide approach & deep approach (may overlapping)
input_A = keras.layers.Input(shape=[5], name="wide_input") # features 0-4
input_B = keras.layers.Input(shape=[6], name="deep_input") # features 2-7
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model = keras.Model(inputs=[input_A, input_B], outputs=[output])

model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))

X_train_A, X_train_B = X_train[:, :5], X_train[:, -6:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, -6:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, -6:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid))

mse_test = model.evaluate((X_test_A, X_test_B), y_test)
y_pred = model.predict((X_new_A, X_new_B))

#### Multiple outputs

In [ ]:
# Add auxiliary output for regularization
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)
model = keras.models.Model(inputs=[input_A, input_B],
                           outputs=[output, aux_output])

# Each output will need its own loss function.
'''
#We should pass a list of losses (if we pass a single loss, 
Keras will assume that the same loss must be used for all outputs). By default,
Keras will compute all these losses and simply add them up to get the final
loss used for training.
'''

model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(lr=1e-3))

history = model.fit([X_train_A, X_train_B], [y_train, y_train], epochs=20,
                    validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]))

total_loss, main_loss, aux_loss = model.evaluate(
    [X_test_A, X_test_B], [y_test, y_test])
y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])


In [ ]:
# Subclassing API
'''
The big difference is that you can do pretty much anything you want in the call() method: 
for loops, if statements, low-level TensorFlow operations. This makes it a great API for researchers
experimenting with new ideas.

This extra flexibility does come at a cost: your model’s architecture is
hidden within the call() method, so Keras cannot easily inspect it; it
cannot save or clone it; and when you call the summary() method, you
only get a list of layers, without any information on how they are
connected to each other. Moreover, Keras cannot check types and shapes
ahead of time, and it is easier to make mistakes. So unless you really need
that extra flexibility, you should probably stick to the Sequential API or
the Functional API.

Use save_weights() and load_weights()
'''

class WideAndDeepModel(keras.models.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
        
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

model = WideAndDeepModel(30, activation="relu")

model.compile(loss="mse", loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(lr=1e-3))
history = model.fit((X_train_A, X_train_B), (y_train, y_train), epochs=10,
                    validation_data=((X_valid_A, X_valid_B), (y_valid, y_valid)))
total_loss, main_loss, aux_loss = model.evaluate((X_test_A, X_test_B), (y_test, y_test))
y_pred_main, y_pred_aux = model.predict((X_new_A, X_new_B))

### Fine-tune models

In [ ]:
# Randomized Search
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)
# keras_reg.fit(X_train, y_train, epochs=100,
#               validation_data=(X_valid, y_valid),
#               callbacks=[keras.callbacks.EarlyStopping(patience=10)])

# mse_test = keras_reg.score(X_test, y_test)
# y_pred = keras_reg.predict(X_new)

from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2),
}

# Note that RandomizedSearchCV uses K-fold crossvalidation, so it does not use 
# X_valid and y_valid, which are only used for early stopping.
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

rnd_search_cv.best_params_
rnd_search_cv.best_score_
model = rnd_search_cv.best_estimator_.model